# x509 Certificate Fingerprinting In Golang
> A tutorial with comparisons to OpenSSL commands.

- toc: true 
- badges: true
- comments: true
- categories: [go, crypto]
- image: /images/copied_from_nb/images/cert_viewer.png

This website serves content to your browser over an encrypted and authenticated tunnel. The **S** in `https` stands for secure via TLS (Transport Layer Security). To do so, it relies on a public key certificate associated with the domain name and issued by a certificate authority (CA) which acts as a trusted third party. I'm using github pages to host this page. Looking at the certificate in Brave's certificate viewer, you see github pages uses lets encrypt as the CA. (If you are looking at this tutorial after November 4th, 2020, you'll see different values.)

![View from Brave's Certificate Viewer](images/cert_viewer.png)

Looking at the same viewer, you'll also notice that Brave shows two fingerprints: one for the (no longer secure) SHA-1 algorithm and the other for the SHA256, a 512 bit block hash in the SHA-2 family. If you aren't doing much work with public key cryptography, this may be the first time you noticed the fingerprint section. However, if you use computers in any sophisticated capacity, you've probably probably have seen hashes of public key artifacts in other contexts. In particular, you may have seen a message like this, 

```
The authenticity of host '<host>' can't be established.
ECDSA key fingerprint is SHA256:<BASE64-encoded-SHA256-Fingerprint>.
Are you sure you want to continue connecting (yes/no)? yes
Warning: Permanently added '<host>' (ECDSA) to the list of known hosts.
``` 

when connecting to a new SSH host using OpenSSH. This user experience is called TOFU: Trust on First Use. Since SSH has not seen this key before (or rather, it's fingerprint) it prompts you whether or not you would like to trust it. If you confirm with `y`, future sessions won't prompt you again, so long as the fingerprint of the key does not change. 

While [TOFU is a questionable practice](https://smallstep.com/blog/use-ssh-certificates/) with respect to security (how often do you actually confirm the fingerprint with an administrator or in some database before confirmation?), this use of fingerprinting is common in applications using TLS. This notebook shows you how to use both OpenSSL and golang to extract equivalent fingerprints. It is fully self-contained — if you have a Golang kernel for Jupyter, you can run it.

## Gold Values from OpenSSL and a PEM-encoded File

First, we'll set up our imports. If you do any public key cryptography in golang, most of these will be familiar.

In [1]:
import (
    "crypto/md5"
    "crypto/sha1"
    "crypto/sha256"
    "crypto/tls"
    "crypto/x509"
    "encoding/pem"
    "fmt"
    "hash"
    "os"
    "os/exec"
    "io/ioutil"
    "strings"
)

Next, we'll write a fixed, minimal ASCII/PEM-encoded certificate so that this tutorial won't change over time.

In [2]:
const knownCert = `-----BEGIN CERTIFICATE-----
MIIBUzCB+qADAgECAhR7l0x6Cgyt0hWRxQXKDB4NIgKM2TAKBggqhkjOPQQDAjAN
MQswCQYDVQQGEwJVSzAeFw0yMDA4MTIyMzIwMTBaFw0zMDA4MTAyMzIwMTBaMA0x
CzAJBgNVBAYTAlVLMFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEJBJm0831oqwg
6daZcuB+vgeRFinuuT6hU2NxboDbfeyGUjv91lBWnCU1YL27d7PhZVabyNtQm0OZ
bmveMV9v7aM4MDYwDgYDVR0PAQH/BAQDAgKEMBMGA1UdJQQMMAoGCCsGAQUFBwMB
MA8GA1UdEwEB/wQFMAMBAf8wCgYIKoZIzj0EAwIDSAAwRQIgWawEXk36iBObsLyr
ZPzIlIN91/85aix5kYRxAPDIkOUCIQDxsqXNpb6hFTiwLnj2stl2hwcxtcTB8bnS
gniTVdxCTw==
-----END CERTIFICATE-----
`

const certFile = "test.crt"
err := ioutil.WriteFile(certFile, []byte(knownCert), os.ModePerm)
if err != nil {
    panic(err)
}

With a certificate in hand (on disk), we can use `openssl` to compute the fingerprints. [OpenSSL](https://www.openssl.org/) is robust and enjoys the battle-tested upside of [Linus's Law](https://en.wikipedia.org/wiki/Linus%27s_law), so it's a useful way to compute the expected outputs using MD5, SHA1, SHA256.

In [8]:
// Extract the fingerprint from command execution in a generic way
func extractFingerprint(cmd *exec.Cmd) string {
    b, err := cmd.Output()
    if err != nil {
        panic(err)
    }
    return strings.Split(strings.TrimSpace(string(b)), "=")[1]
}

knownMD5 := extractFingerprint(exec.Command("openssl", "x509", "-noout", "-fingerprint", "-md5", "-in", certFile))
knownSHA1 := extractFingerprint(exec.Command("openssl", "x509", "-noout", "-fingerprint", "-sha1", "-in", certFile))
knownSHA256 := extractFingerprint(exec.Command("openssl", "x509", "-noout", "-fingerprint", "-sha256", "-in", certFile))
_, _ := fmt.Printf("MD5    = %s\nSHA1   = %s\nSHA256 = %s\n", knownMD5, knownSHA1, knownSHA256)

MD5    = 04:56:1B:6C:AF:DE:39:72:18:74:AE:E0:F9:5B:2D:DE
SHA1   = 3A:1F:3B:C0:B3:A4:5B:FE:D6:D7:87:5E:C7:1D:E4:C4:7B:EA:B0:C0
SHA256 = D7:AD:9F:D5:C4:F4:BD:23:7B:DE:BF:7F:30:C0:D8:99:7A:C8:72:94:DE:DA:25:C7:4E:6C:3B:06:C4:EB:4E:C0


## Fingerprinting without x509 Parsing

With the gold values computed, we can now use Go, instead. First, we'll load the certificate file and decode the PEM data.

In [4]:
b, err := ioutil.ReadFile(certFile)
if err != nil {
    panic(err)
}

block, _ := pem.Decode(b)
if block == nil {
    panic("doesn't seem like a PEM block")
}

Now, we'll compute the MD5, SHA1, and SHA256, hashes from the PEM block's `Bytes` field.

In [5]:
// Normalize the hash so it is equal to the openssl output
func normalizeHash(h hash.Hash, b []byte) string {
    _, err := h.Write(b)
    if err != nil {
        panic(err)
    }
    digest := h.Sum([]byte{})
                         
    var parts []string
    for _, octet := range digest {
        parts = append(parts, fmt.Sprintf("%02X", octet))
    }
    return strings.Join(parts, ":")
}

_, _ := fmt.Printf("MD5 Equal    = %v\nSHA1 Equal   = %v\nSHA256 Equal = %v\n",
    normalizeHash(md5.New(), block.Bytes) == knownMD5,
    normalizeHash(sha1.New(), block.Bytes) == knownSHA1,
    normalizeHash(sha256.New(), block.Bytes) == knownSHA256,
)

MD5 Equal    = true
SHA1 Equal   = true
SHA256 Equal = true


They are equal. The go code and the OpenSSL commands compute the same thing.

## Fingerprinting with x509 Parsing

This part is a little contrived. But, often you have ready access to a `*x509.Certificate`. If you do, the `Raw` field contains the same data as the block bytes,

In [6]:
cert, err := x509.ParseCertificate(block.Bytes)
if block == nil {
    panic("doesn't seem like a PEM block")
}

_, _ := fmt.Printf("MD5 Equal    = %v\nSHA1 Equal   = %v\nSHA256 Equal = %v\n",
    normalizeHash(md5.New(), cert.Raw) == knownMD5,
    normalizeHash(sha1.New(), cert.Raw) == knownSHA1,
    normalizeHash(sha256.New(), cert.Raw) == knownSHA256,
)

MD5 Equal    = true
SHA1 Equal   = true
SHA256 Equal = true


## Fingerprinting from a Peer Certificate

Finally, most code that uses `net` makes the peer certificates for an active connection available in some way. This example shows how to take the fingerprint of that certificate.

In [10]:
conn, err := tls.Dial("tcp", "code.johnbnelson.com:443", &tls.Config{})
if err != nil {
    panic(err)
}

// There may be more than one certificate. This example may break.
fetchedCert := conn.ConnectionState().PeerCertificates[0]
conn.Close()
normalizeHash(sha256.New(), fetchedCert.Raw)

D7:AB:C0:10:83:65:B2:59:CA:88:5B:94:43:EF:86:E5:49:B6:F0:57:38:4F:4F:33:BA:27:8C:DB:D6:D2:88:2B

## Conclusion

Go is batteries included from the perspective of public key cryptography. The most complicated part of this tutorial was string manipulation, which you don't have to actually do. The only *general* parting advice is, as always, don't use MD5 or SHA1 — they are insecure.